# Импорт необходимых библиотек

In [60]:
import pandas as pd

# Загружаем датафреймы

In [61]:
df_test = pd.read_csv('/home/data_srv_admin/projects/2_MLOPS/data/RAW/test.csv', index_col='PassengerId')

In [87]:
df_test.head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
PassengerId,,,,,,,,
892,3,1,34.5,0,0,7.8292,0,1
893,3,0,47.0,1,0,7.0000,0,2
894,2,1,62.0,0,0,9.6875,0,1
895,3,1,27.0,0,0,8.6625,0,2
896,3,0,22.0,1,1,12.2875,0,2


In [63]:
df_train = pd.read_csv('/home/data_srv_admin/projects/2_MLOPS/data/RAW/train.csv', index_col='PassengerId')

In [88]:
df_train.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
1,0,3,1,22.0,1,0,7.2500,0,2
2,1,1,0,38.0,1,0,71.2833,1,0
3,1,3,0,26.0,0,0,7.9250,0,2
4,1,1,0,35.0,1,0,53.1000,1,2
5,0,3,1,35.0,0,0,8.0500,0,2


**Краткий вывод: в исследуемом датасете имеется 10 входных признаков и 1 целевая метка (будем решать задачу классификации)**

# Чистка и заполнение

In [65]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, 892 to 1309
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Name      418 non-null    object 
 2   Sex       418 non-null    object 
 3   Age       332 non-null    float64
 4   SibSp     418 non-null    int64  
 5   Parch     418 non-null    int64  
 6   Ticket    418 non-null    object 
 7   Fare      417 non-null    float64
 8   Cabin     91 non-null     object 
 9   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 35.9+ KB


Всего 418 значений, отсутствует часть данных о возрасте, сильно пострадал параметр Cabin, поэтому, возможно, от него придётся избавиться

In [66]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


Картина та же, что и выше. Думаем над моделью заполнения пропусков для Возраста и целесообразность фичи Cabin в целом, а при её необходимости - над заполнением пропусков.

# Общий вывод:

Параметр Age заполняю медианой, а Cabin - 0, если НаН, 1 - если не НаН. Остальные NaNы дропаю.

# Проверяем

In [67]:
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())

In [68]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       891 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


то же самое для тестовой выборки

In [69]:
df_test['Age'] = df_test['Age'].fillna(df_test['Age'].median())

In [70]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, 892 to 1309
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Name      418 non-null    object 
 2   Sex       418 non-null    object 
 3   Age       418 non-null    float64
 4   SibSp     418 non-null    int64  
 5   Parch     418 non-null    int64  
 6   Ticket    418 non-null    object 
 7   Fare      417 non-null    float64
 8   Cabin     91 non-null     object 
 9   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 35.9+ KB


Далее заполняем пропуски в параметре Cabin как определялись выше

In [71]:
df_test['Cabin'] = df_test['Cabin'].fillna(0)

In [72]:
df_train['Cabin'] = df_train['Cabin'].fillna(0)

In [73]:
df_test.Cabin[df_test.Cabin != 0] = 1

/tmp/ipykernel_3176/216283.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.Cabin[df_test.Cabin != 0] = 1


In [74]:
df_train.Cabin[df_train.Cabin != 0] = 1

/tmp/ipykernel_3176/1292743878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.Cabin[df_train.Cabin != 0] = 1


Какие столбцы нам не нужны

In [75]:
df_test.drop(columns=['Name', 'Ticket'], inplace=True)

In [76]:
df_train.drop(columns=['Name', 'Ticket'], inplace=True)

Переводим в булевы значения пол и другие фичи

In [83]:
df_test.Sex = df_test.Sex.astype('category').cat.codes

In [85]:
df_test.Embarked = df_test.Embarked.astype('category').cat.codes

In [82]:
df_train.Sex = df_train.Sex.astype('category').cat.codes

In [86]:
df_train.Embarked = df_train.Embarked.astype('category').cat.codes

In [89]:
for column in df_train.columns:
    df_train[column] = df_train[column].fillna(df_train[column].median())